In [53]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

#0. Mode 설정

In [54]:
mode = True
# mode = False

# bracket_pricing별로 구분한다
# bracket = 'Yes'
bracket = 'No'
# bracket = 'All'

#0. Dataset 읽기

In [55]:
import pandas as pd
import numpy as np

df_train = pd.read_csv( './dataset/train_set.csv' )    # data를 읽어들인다.
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

In [56]:
if bracket == 'Yes' :
    # bracket_pricing == Yes인 경우만
    df_train = df_train[ df_train[ 'bracket_pricing' ] == 'Yes' ]
elif bracket == 'No' :
    # bracket_pricing == No인 경우만
    df_train = df_train[ df_train[ 'bracket_pricing' ] == 'No' ]

#1. Feature 추출 function
* quote_date 활용하여 year, month, day 확보

In [57]:
import datetime

def extract_year_month_day_from_quote_date( p_df ) :    # quote date로부터 연도, 월, 날짜를 추출
    p_df[ 'quote_date' ] = pd.to_datetime( p_df[ 'quote_date' ] )    # string을 datetime으로 형변환
    p_df[ 'year' ] = p_df[ 'quote_date' ].dt.year    # 연도
    p_df[ 'month' ] = p_df[ 'quote_date' ].dt.month    # 월
    p_df[ 'day' ] = p_df[ 'quote_date' ].dt.day    # 일
    
    return p_df

In [58]:
def make_string_date( p_df ) :    # 일자정보를 string으로 변경
    y = p_df[ 'year' ]
    m = p_df[ 'month' ]
    d = p_df[ 'day' ]
    
    str_y = ''
    str_m = ''
    str_d = ''
    
    str_y = str( y )
    str_m = ''
    if m < 10 :
        str_m = '0' + str( m )
    else :
        str_m = str( m )
    str_d = ''
    if d < 10 :
        str_d = '0' + str( d )
    else :
        str_d = str( d )
    
    return str_y + str_m + str_d

#2. Label Encoding function
* 현재, train/ test dataset과 tube dataset과 merge하는 case까지 반영하여 encoder 관리

In [59]:
df_train.count()

tube_assembly_id      3930
supplier              3930
quote_date            3930
annual_usage          3930
min_order_quantity    3930
bracket_pricing       3930
quantity              3930
cost                  3930
dtype: int64

In [60]:
from sklearn import preprocessing

# le_bracket_pricing = preprocessing.LabelEncoder()
le_supplier = preprocessing.LabelEncoder()
le_material_id = preprocessing.LabelEncoder()
le_date = preprocessing.LabelEncoder()

le_yn = preprocessing.LabelEncoder()
le_yn.fit( [ 'Y', 'N' ] )


le_end = preprocessing.LabelEncoder()
df_end = pd.read_csv( './dataset/01.original.dataset/tube_end_form.csv' )
le_end.fit( df_end[ 'end_form_id' ] )
df_end = ''


le_component_id = preprocessing.LabelEncoder()    # component의 경우, master dataset이 별도로 존재하므로
df_components = pd.read_csv( './dataset/02.ml.verified.dataset/components.verified.csv' )
le_component_id.fit( df_components[ 'component_id' ] )


le_spec_id = preprocessing.LabelEncoder()    # spec의 경우, spec meta dataset을 별도로 생성했음
df_spec_meta = pd.read_csv( './dataset/01.original.dataset/spec_meta.csv' )
le_spec_id.fit( df_spec_meta[ 'spec' ] )
df_spec_meta = ''


le_component_mac475 = preprocessing.LabelEncoder()
le_component_mac475.fit( df_components[ 'component_mac475' ] )


le_component_type_id = preprocessing.LabelEncoder()
le_component_type_id.fit( df_components[ 'component_type_id' ] )
df_components = ''


le_type_end = preprocessing.LabelEncoder()
df_type_end = pd.read_csv( './dataset/01.original.dataset/type_end_form.csv' )
le_type_end.fit( df_type_end[ 'end_form_id' ] )
df_type_end = ''


le_connection = preprocessing.LabelEncoder()
df_connection = pd.read_csv( './dataset/01.original.dataset/type_connection.csv' )
le_connection.fit( df_connection[ 'connection_type_id' ] )
df_connection = ''


le_yn_none = preprocessing.LabelEncoder()
le_yn_none.fit( [ 'Y', 'N', 'NONE' ] )


le_boss_type = preprocessing.LabelEncoder()
le_boss_type.fit( [ 'Boss', 'Stud', 'NONE' ] )


le_elbow_mj_class_code = preprocessing.LabelEncoder()
le_elbow_mj_class_code.fit( [ 'N', 'MJ-001', 'MJ-003', 'MJ-005', 'MJ-006', 'NONE' ] )

le_boss_outside_shape = preprocessing.LabelEncoder()
le_boss_outside_shape.fit( [ 'Round', 'Hex', 'NONE' ] )


le_boss_base_type = preprocessing.LabelEncoder()
le_boss_base_type.fit( [ 'Flat Bottom', 'Saddle', 'Shoulder', 'NONE' ] )

def executeLabelEncoding( p_df, is_init ) :
    if is_init == True :    # training dataset인 경우, label encoder 생성 및 fitting 수행
        p_df[ 'supplier' ] = le_supplier.fit_transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.fit_transform( p_df[ 'material_id' ] )
        p_df[ 'date' ] = le_date.fit_transform( p_df[ 'date' ] )
    else :
        p_df[ 'supplier' ] = le_supplier.transform( p_df[ 'supplier' ] )
        p_df[ 'material_id' ] = le_material_id.transform( p_df[ 'material_id' ] )
        p_df[ 'date' ] = le_date.transform( p_df[ 'date' ] )
        
    # 무조건 수행
    p_df[ 'is_annual' ] = le_yn.transform( p_df[ 'is_annual' ] )
    p_df[ 'bracket_pricing' ] = le_yn.transform( p_df[ 'bracket_pricing' ] )
    
    # tube/material_id == null을 tube만 사용예측시 적용, bill/ spec 같이 사용예측시 주석처리
#     p_df[ 'end_a_1x' ] = le_yn.transform( p_df[ 'end_a_1x' ] )
#     p_df[ 'end_a_2x' ] = le_yn.transform( p_df[ 'end_a_2x' ] )
#     p_df[ 'end_x_1x' ] = le_yn.transform( p_df[ 'end_x_1x' ] )
#     p_df[ 'end_x_2x' ] = le_yn.transform( p_df[ 'end_x_2x' ] )
    
    p_df[ 'end_a' ] = le_end.transform( p_df[ 'end_a' ] )
    p_df[ 'end_x' ] = le_end.transform( p_df[ 'end_x' ] )

    for i in range( 1, 9 ) :    # bill_of_materials에서 merge된 component_id_1~8을 label encoding 수행
        comp_str = 'component_id_' + str( i )
        p_df[ comp_str ] = le_component_id.transform( p_df[ comp_str ] )

    for i in range( 1, 11 ) :    # specs에서 merge된 spec1~10을 label encoding 수행
        spec_str = 'spec' + str( i )
        p_df[ spec_str ] = le_spec_id.transform( p_df[ spec_str ] )                
        
    p_df[ 'forming_x' ] = le_yn.transform( p_df[ 'forming_x' ] )
    p_df[ 'forming_y' ] = le_yn.transform( p_df[ 'forming_y' ] )
    
    
    comp_list = [ 'x' ]
    for v in comp_list :
        v = ''
        p_df[ 'component_type_id' + v ] = le_component_type_id.transform( p_df[ 'component_type_id' + v ] )
        p_df[ 'component_mac475' + v ] = le_component_mac475.transform( p_df[ 'component_mac475' + v ] )


        p_df[ 'adaptor_end_form_id_1' + v ] = le_type_end.transform( p_df[ 'adaptor_end_form_id_1' + v ] )
        p_df[ 'adaptor_connection_type_id_1' + v ] = le_connection.transform( p_df[ 'adaptor_connection_type_id_1' + v ] )
        p_df[ 'adaptor_end_form_id_2' + v ] = le_type_end.transform( p_df[ 'adaptor_end_form_id_2' + v ] )
        p_df[ 'adaptor_connection_type_id_2' + v ] = le_connection.transform( p_df[ 'adaptor_connection_type_id_2' + v ] )
        p_df[ 'adaptor_unique_feature' + v ] = le_yn_none.transform( p_df[ 'adaptor_unique_feature' + v ] )
        p_df[ 'adaptor_orientation' + v ] = le_yn_none.transform( p_df[ 'adaptor_orientation' + v ] )


        p_df[ 'boss_type' + v ] = le_boss_type.transform( p_df[ 'boss_type' + v ] )
        p_df[ 'boss_connection_type_id' + v ] = le_connection.transform( p_df[ 'boss_connection_type_id' + v ] )
        p_df[ 'boss_outside_shape' + v ] = le_boss_outside_shape.transform( p_df[ 'boss_outside_shape' + v ] )
        p_df[ 'boss_base_type' + v ] = le_boss_base_type.transform( p_df[ 'boss_base_type' + v ] )
        p_df[ 'boss_groove' + v ] = le_yn_none.transform( p_df[ 'boss_groove' + v ] )
        p_df[ 'boss_unique_feature' + v ] = le_yn_none.transform( p_df[ 'boss_unique_feature' + v ] )
        p_df[ 'boss_orientation' + v ] = le_yn_none.transform( p_df[ 'boss_orientation' + v ] )


        p_df[ 'elbow_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'elbow_mj_class_code' + v ] )
        p_df[ 'elbow_mj_plug_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'elbow_mj_plug_class_code' + v ] )
        p_df[ 'elbow_groove' + v ] = le_yn_none.transform( p_df[ 'elbow_groove' + v ] )
        p_df[ 'elbow_unique_feature' + v ] = le_yn_none.transform( p_df[ 'elbow_unique_feature' + v ] )
        p_df[ 'elbow_orientation' + v ] = le_yn_none.transform( p_df[ 'elbow_orientation' + v ] )

        p_df[ 'float_orientation' + v ] = le_yn_none.transform( p_df[ 'float_orientation' + v ] )

        p_df[ 'hfl_corresponding_shell' + v ] = le_yn_none.transform( p_df[ 'hfl_corresponding_shell' + v ] )
        p_df[ 'hfl_coupling_class' + v ] = le_yn_none.transform( p_df[ 'hfl_coupling_class' + v ] )
        p_df[ 'hfl_material' + v ] = le_yn_none.transform( p_df[ 'hfl_material' + v ] )
        p_df[ 'hfl_plating' + v ] = le_yn_none.transform( p_df[ 'hfl_plating' + v ] )
        p_df[ 'hfl_orientation' + v ] = le_yn_none.transform( p_df[ 'hfl_orientation' + v ] )

        p_df[ 'nut_blind_hole' + v ] = le_yn_none.transform( p_df[ 'nut_blind_hole' + v ] )
        p_df[ 'nut_orientation' + v ] = le_yn_none.transform( p_df[ 'nut_orientation' + v ] )

        p_df[ 'sleeve_connection_type_id' + v ] = le_connection.transform( p_df[ 'sleeve_connection_type_id' + v ] )    
        p_df[ 'sleeve_unique_feature' + v ] = le_yn_none.transform( p_df[ 'sleeve_unique_feature' + v ] )
        p_df[ 'sleeve_plating' + v ] = le_yn_none.transform( p_df[ 'sleeve_plating' + v ] )
        p_df[ 'sleeve_orientation' + v ] = le_yn_none.transform( p_df[ 'sleeve_orientation' + v ] )


        p_df[ 'straight_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'straight_mj_class_code' + v ] )
        p_df[ 'straight_groove' + v ] = le_yn_none.transform( p_df[ 'straight_groove' + v ] )
        p_df[ 'straight_unique_feature' + v ] = le_yn_none.transform( p_df[ 'straight_unique_feature' + v ] )
        p_df[ 'straight_orientation' + v ] = le_yn_none.transform( p_df[ 'straight_orientation' + v ] )


        p_df[ 'tee_mj_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'tee_mj_class_code' + v ] )
        p_df[ 'tee_mj_plug_class_code' + v ] = le_elbow_mj_class_code.transform( p_df[ 'tee_mj_plug_class_code' + v ] )
        p_df[ 'tee_groove' + v ] = le_yn_none.transform( p_df[ 'tee_groove' + v ] )    
        p_df[ 'tee_unique_feature' + v ] = le_yn_none.transform( p_df[ 'tee_unique_feature' + v ] )    
        p_df[ 'tee_orientation' + v ] = le_yn_none.transform( p_df[ 'tee_orientation' + v ] )    

        p_df[ 'threaded_end_form_id_1' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_1' + v ] )
        p_df[ 'threaded_connection_type_id_1' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_1' + v ] )    
        p_df[ 'threaded_end_form_id_2' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_2' + v ] )
        p_df[ 'threaded_connection_type_id_2' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_2' + v ] )
        p_df[ 'threaded_end_form_id_3' + v ] = le_type_end.transform( p_df[ 'threaded_end_form_id_3' + v ] )
        p_df[ 'threaded_connection_type_id_3' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_3' + v ] )
        p_df[ 'threaded_end_form_id_4' + v ] = le_end.transform( p_df[ 'threaded_end_form_id_4' + v ] )
        p_df[ 'threaded_connection_type_id_4' + v ] = le_connection.transform( p_df[ 'threaded_connection_type_id_4' + v ] )
        p_df[ 'threaded_unique_feature' + v ] = le_yn_none.transform( p_df[ 'threaded_unique_feature' + v ] )
        p_df[ 'threaded_orientation' + v ] = le_yn_none.transform( p_df[ 'threaded_orientation' + v ] )

    return p_df

.

#3. Label Encoding용 Dataset
* categorical data를 모두 포함해야 하므로, tube join후 기반으로 label encoder를 fitting한다
* label encoding의 대상은 bracket_pricing, supplier, material_id, end_a_1x 시리즈, end_a 시리즈 feature이다

##3.1 train/ test dataset merge

In [61]:
df_train[ 'id' ] = 99999    # test와 join위해 feature 추가 : 99999는 train dataset이다

In [62]:
df_merged = df_train.append( df_test )    # train과 test df를 merge한다.

##3.1 tube_bill_specs dataset merge 및 label encoding

In [63]:
df_tube_bill_specs_end_comp = pd.read_csv( './dataset/03.merged/tube_bill_specs_end_comp_merged.csv' )
# df_tube_bill_specs_end_comp = pd.read_csv( './dataset/03.merged/tube_bill_specs_end_merged.csv' )

In [64]:
df_merged = df_merged.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

In [65]:
dic = { 'Yes' : 'Y', 'No' : 'N' }
df_merged[ 'bracket_pricing' ].replace( dic, inplace = True )

In [66]:
def get_is_annual( p_df ) :    # annual_usage 존재여부를 확인
    is_annual = 'N'
    if p_df[ 'annual_usage' ] > 0 :
        is_annual = 'Y'
    return is_annual

In [67]:
df_merged[ 'is_annual' ] = df_merged.apply( get_is_annual, axis = 1 )

In [68]:
# min_order_quantity 조건테스트
df_merged[ 'min_order_quantity' ].replace( to_replace = 0, value = 1, inplace = True )

In [69]:
def get_order_count( p_df, p_type ) :    # 주문횟수를 계산
    order_count = 0
    
    if p_type == 'min' :
        if ( p_df[ 'annual_usage' ] > 0 ) & ( p_df[ 'min_order_quantity' ] > 0 ) :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'min_order_quantity' ] )
    elif p_type == 'quantity' :
        order_count = float( p_df[ 'annual_usage' ] / p_df[ 'quantity' ] )
    elif p_type == 'rule' :
        if p_df[ 'bracket_pricing' ] == 'Y' :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'quantity' ] )
        elif p_df[ 'bracket_pricing' ] == 'N' :
            order_count = float( p_df[ 'annual_usage' ] / p_df[ 'min_order_quantity' ] )
    elif p_type == 'sales' :
        order_count = float( p_df[ 'annual_usage' ] * p_df[ 'quantity' ] )
            
    return order_count

In [70]:
df_merged[ 'order_count_by_min' ] = df_merged.apply( get_order_count, axis = 1, args = ('min',) )
df_merged[ 'order_count_by_quantity' ] = df_merged.apply( get_order_count, axis = 1, args = ('quantity',) )
df_merged[ 'order_count_by_rule' ] = df_merged.apply( get_order_count, axis = 1, args = ('rule',) )
df_merged[ 'sales_result' ] = df_merged.apply( get_order_count, axis = 1, args = ('sales',) )
df_merged = extract_year_month_day_from_quote_date( df_merged )    # feature 처리를 수행

In [71]:
df_merged[ 'date' ] = df_merged.apply( make_string_date, axis = 1 )

In [72]:
df_merged = executeLabelEncoding( df_merged, True )

In [73]:
df_merged.columns

Index(['annual_usage', 'bracket_pricing', 'cost', 'id', 'min_order_quantity', 'quantity', 'quote_date', 'supplier', 'tube_assembly_id', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other', 'bend_num_by_radius', 'component_id_1', 'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8', 'weight_sum', 'uniqueness_count', 'orientation_count', 'adaptor_sum', 'boss_sum', 'elbow_sum', 'float_sum', 'hfl_sum', 'nut_sum', 'other_sum', 'sleeve_sum', 'straight_sum', 'tee_sum', 'threaded_sum', 'adaptor_weight_sum', 'boss_weight_sum', 'elbow_weight_sum', 'float_weight_sum', 'hfl_weight_sum', 'nut_weight_sum', 'other_weight_sum', 'sleeve_weight_sum', 'straight_weight_sum', 'tee_weight_sum', 'threaded_we

In [74]:
df_merged.to_csv( './dataset/df_train_test_tube_bill_specs_end_comp_merged_encoded.csv', index = False )

In [75]:
df_train.drop( 'id', axis = 1, inplace = True )    # 일단, 필요없는 것들 제거 : df_train을 원상 복구해둠
del( df_test )    # 일단, 필요없는 것들 제거
del( df_merged )    # 일단, 필요없는 것들 제거

#4. Modeling

##4.1 Merged features중, modeling시 제거할 feature 관리

In [76]:
list_for_remove = []    # 전역변수의 선언
def executeFeatureRemoval( p_df ) :
    # 제거하고자 하는 feature list
    global list_for_remove    # 전역변수 명시
    
    list_for_remove = [
                        'tube_assembly_id',
                        'quote_date',
#                         'date',

#                         'supplier',        
#                         'annual_usage',
#                         'min_order_quantity',
#                         'bracket_pricing',
        
#                         'order_count_by_quantity',
#                         'order_count_by_min',
#                         'order_count_by_rule',
#                         'sales_result',
        
        
#                         'quantity',
#                         'material_id',
#                         'diameter',
#                         'wall',
#                         'length',
#                         'num_bends',
#                         'bend_radius',
#                         'end_a_1x',
#                         'end_a_2x',
#                         'end_x_1x',
#                         'end_x_2x',
#                         'end_a',
#                         'end_x',
#                         'num_boss',
#                         'num_bracket',
#                         'other',
        
#                         'component_id_1',
#                         'quantity_1',
#                         'component_id_2',
#                         'quantity_2',
#                         'component_id_3',
#                         'quantity_3',
#                         'component_id_4',
#                         'quantity_4',

#                         'component_id_5',
#                         'quantity_5',
#                         'component_id_6',
#                         'quantity_6',
#                         'component_id_7',
#                         'quantity_7',
#                         'component_id_8',
#                         'quantity_8',
        
#                         'weight_sum',
#                         'adaptor_sum',
#                         'boss_sum',
#                         'elbow_sum',
#                         'float_sum',
#                         'hfl_sum',
#                         'nut_sum',
#                         'other_sum',
#                         'sleeve_sum',
#                         'straight_sum',
#                         'tee_sum',
#                         'threaded_sum',
        
#                         'adaptor_weight_sum',
#                         'boss_weight_sum',
#                         'elbow_weight_sum',
#                         'float_weight_sum',
#                         'hfl_weight_sum',
#                         'nut_weight_sum',
#                         'other_weight_sum',
#                         'sleeve_weight_sum',
#                         'straight_weight_sum',
#                         'tee_weight_sum',
#                         'threaded_weight_sum',
        
#                         'comp_type_count',
#                         'comp_total_count',
#                         'tube_volume',
        
#                         'spec1',
#                         'spec2',
#                         'spec3',
#                         'spec4',
#                         'spec5',
        
#                         'spec6',
#                         'spec7',
#                         'spec8',
#                         'spec9',
#                         'spec10',
        
#                         'spec_type_count',
#                         'forming_x',
#                         'forming_y',

                        # component family 계열의 merge를 하지않은 case에 대한 feature들을 일단 죽여둔다
                        'component_type_id',
                        'mac475_weight',
#                         'component_mac475',
                        
                        'uniqueness',
                        'orientation',
        
                        'adaptor_adaptor_angle',
                        'adaptor_overall_length',
                        'adaptor_end_form_id_1',
                        'adaptor_connection_type_id_1',
                        'adaptor_length_1',
                        'adaptor_thread_size_1',
                        'adaptor_thread_pitch_1',
                        'adaptor_nominal_size_1',
                        'adaptor_end_form_id_2',
                        'adaptor_connection_type_id_2',
                        'adaptor_length_2',
                        'adaptor_thread_size_2',
                        'adaptor_thread_pitch_2',
                        'adaptor_nominal_size_2',
                        'adaptor_hex_size',
                        'adaptor_unique_feature',
                        'adaptor_orientation',
                        'adaptor_weight',

                        'boss_type',
                        'boss_connection_type_id',
                        'boss_outside_shape',
                        'boss_base_type',
                        'boss_height_over_tube',
                        'boss_bolt_pattern_long',
                        'boss_bolt_pattern_wide',
                        'boss_groove',
                        'boss_base_diameter',
                        'boss_shoulder_diameter',
                        'boss_unique_feature',
                        'boss_orientation',
                        'boss_weight',

                        'elbow_bolt_pattern_long',
                        'elbow_bolt_pattern_wide',
                        'elbow_extension_length',
                        'elbow_overall_length',
                        'elbow_thickness',
                        'elbow_drop_length',
                        'elbow_elbow_angle',
                        'elbow_mj_class_code',
                        'elbow_mj_plug_class_code',
                        'elbow_plug_diameter',
                        'elbow_groove',
                        'elbow_unique_feature',
                        'elbow_orientation',
                        'elbow_weight',

                        'float_bolt_pattern_long',
                        'float_bolt_pattern_wide',
                        'float_thickness',
                        'float_orientation',
                        'float_weight',

                        'hfl_hose_diameter',
                        'hfl_corresponding_shell',
                        'hfl_coupling_class',
                        'hfl_material',
                        'hfl_plating',
                        'hfl_orientation',
                        'hfl_weight',

                        'nut_hex_nut_size',
                        'nut_seat_angle',
                        'nut_length',
                        'nut_thread_size',
                        'nut_thread_pitch',
                        'nut_diameter',
                        'nut_blind_hole',
                        'nut_orientation',
                        'nut_weight',

                        'other_weight',

                        'sleeve_connection_type_id',
                        'sleeve_length',
                        'sleeve_intended_nut_thread',
                        'sleeve_intended_nut_pitch',
                        'sleeve_unique_feature',
                        'sleeve_plating',
                        'sleeve_orientation',
                        'sleeve_weight',

                        'straight_bolt_pattern_long',
                        'straight_bolt_pattern_wide',
                        'straight_head_diameter',
                        'straight_overall_length',
                        'straight_thickness',
                        'straight_mj_class_code',
                        'straight_groove',
                        'straight_unique_feature',
                        'straight_orientation',
                        'straight_weight',

                        'tee_bolt_pattern_long',
                        'tee_bolt_pattern_wide',
                        'tee_extension_length',
                        'tee_overall_length',
                        'tee_thickness',
                        'tee_drop_length',
                        'tee_mj_class_code',
                        'tee_mj_plug_class_code',
                        'tee_groove',
                        'tee_unique_feature',
                        'tee_orientation',
                        'tee_weight',

                        'threaded_adaptor_angle',
                        'threaded_overall_length',
                        'threaded_hex_size',
                        'threaded_end_form_id_1',
                        'threaded_connection_type_id_1',
                        'threaded_length_1',
                        'threaded_thread_size_1',
                        'threaded_thread_pitch_1',
                        'threaded_nominal_size_1',
                        'threaded_end_form_id_2',
                        'threaded_connection_type_id_2',
                        'threaded_length_2',
                        'threaded_thread_size_2',
                        'threaded_thread_pitch_2',
                        'threaded_nominal_size_2',
                        'threaded_end_form_id_3',
                        'threaded_connection_type_id_3',
                        'threaded_length_3',
                        'threaded_thread_size_3',
                        'threaded_thread_pitch_3',
                        'threaded_nominal_size_3',
                        'threaded_end_form_id_4',
                        'threaded_connection_type_id_4',
                        'threaded_length_4',
                        'threaded_thread_size_4',
                        'threaded_thread_pitch_4',
                        'threaded_nominal_size_4',
                        'threaded_unique_feature',
                        'threaded_orientation',
                        'threaded_weight',
                      ]

    return p_df.drop( list_for_remove, axis = 1, inplace = False )

##4.2 Train : 연/ 월/ 일 Feature 추출

In [77]:
df_train = extract_year_month_day_from_quote_date( df_train )    # feature 처리를 수행
df_train[ 'date' ] = df_train.apply( make_string_date, axis = 1 )

##4.3 Train : Tube와 merge통한 Feature 확장

In [78]:
df_train_merged = df_train.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

##4.4 Train : Merged dataset Label Encoding 수행

In [79]:
df_train_merged[ 'bracket_pricing' ].replace( dic, inplace = True )

In [80]:
df_train_merged[ 'is_annual' ] = df_train_merged.apply( get_is_annual, axis = 1 )
df_train_merged[ 'order_count_by_min' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('min',) )
df_train_merged[ 'order_count_by_quantity' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('quantity',) )
df_train_merged[ 'order_count_by_rule' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('rule',) )
df_train_merged[ 'sales_result' ] = df_train_merged.apply( get_order_count, axis = 1, args = ('sales',) )

In [81]:
df_train_merged[ 'req_comp_spec_type_count' ] = df_train_merged[ 'spec_type_count' ] + df_train_merged[ 'comp_type_count' ]
df_train_merged[ 'req_comp_spec_total_count' ] = df_train_merged[ 'spec_type_count' ] + df_train_merged[ 'comp_type_count' ] * df_train_merged[ 'comp_total_count' ]

In [82]:
df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged.csv' )

In [83]:
df_train_merged = executeLabelEncoding( df_train_merged, is_init = False )    # label encoding을 수행한다

In [84]:
df_train_merged.to_csv( './dataset/df_train_tube_bill_specs_end_comp_merged_encoded.csv' )

##4.5 Train : 불필요 Feature 제거

In [85]:
df_train_merged = executeFeatureRemoval( df_train_merged )    # feature removal 처리를 수행

.

##4.6 Test : Dataset 읽기

In [86]:
df_test = pd.read_csv( './dataset/test_set.csv' )    # data를 읽어들인다.

In [87]:
if bracket == 'Yes' :
    # bracket_pricing == Yes인 경우만
    df_test = df_test[ df_test[ 'bracket_pricing' ] == 'Yes' ]
elif bracket == 'No' :
    # bracket_pricing == No인 경우만
    df_test = df_test[ df_test[ 'bracket_pricing' ] == 'No' ]

##4.7 Test : 연/ 월/ 일 Feature 추출

In [88]:
df_test = extract_year_month_day_from_quote_date( df_test )    # feature 처리를 수행
df_test[ 'date' ] = df_test.apply( make_string_date, axis = 1 )

df_result = pd.DataFrame( df_test[ 'id' ], columns = ['id'] )    # 결과용 dataframe을 생성
df_test.drop( 'id', axis = 1, inplace = True )    # id feature는 일단 제거

##4.8 Test : Tube와 merge통한 Feature 확장

In [89]:
df_test_merged = df_test.merge( df_tube_bill_specs_end_comp, how = 'inner', on = 'tube_assembly_id' )

##4.9 Test : Merged dataset Label Encoding 수행

In [90]:
df_test_merged[ 'bracket_pricing' ].replace( dic, inplace = True )
df_test_merged[ 'is_annual' ] = df_test_merged.apply( get_is_annual, axis = 1 )
df_test_merged[ 'order_count_by_min' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('min',) )
df_test_merged[ 'order_count_by_quantity' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('quantity',) )
df_test_merged[ 'order_count_by_rule' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('rule',) )
df_test_merged[ 'sales_result' ] = df_test_merged.apply( get_order_count, axis = 1, args = ('sales',) )

df_test_merged[ 'req_comp_spec_type_count' ] = df_test_merged[ 'spec_type_count' ] + df_test_merged[ 'comp_type_count' ]
df_test_merged[ 'req_comp_spec_total_count' ] = df_test_merged[ 'spec_type_count' ] + df_test_merged[ 'comp_type_count' ] * df_test_merged[ 'comp_total_count' ]

df_test_merged = executeLabelEncoding( df_test_merged, is_init = False )    # label encoding을 수행한다

In [91]:
df_test_merged.count()

tube_assembly_id      3956
supplier              3956
quote_date            3956
annual_usage          3956
min_order_quantity    3956
bracket_pricing       3956
quantity              3956
year                  3956
month                 3956
day                   3956
date                  3956
material_id           3956
diameter              3956
wall                  3956
length                3956
...
threaded_connection_type_id_4    3956
threaded_length_4                3956
threaded_thread_size_4           3956
threaded_thread_pitch_4          3956
threaded_nominal_size_4          3956
threaded_unique_feature          3956
threaded_orientation             3956
threaded_weight                  3956
is_annual                        3956
order_count_by_min               3956
order_count_by_quantity          3956
order_count_by_rule              3956
sales_result                     3956
req_comp_spec_type_count         3956
req_comp_spec_total_count        3956
Length: 319, dtype: i

##4.10 Test : 불필요 Featrue 제거

In [92]:
df_test_merged = executeFeatureRemoval( df_test_merged )    # feature removal 처리를 수행

.

In [93]:
# log함수 적용 테스트
# X[ 'tube_volume' ] = np.log10( X[ 'tube_volume' ] )
# df_test_merged[ 'tube_volume' ] = np.log10( df_test_merged[ 'tube_volume' ] )

# bend nub으로 인한 tube의 복잡도 추정 feature의 추가
df_train_merged[ 'bend_num_X_volume' ] = df_train_merged[ 'num_bends' ].replace( 0, 1 ) * df_train_merged[ 'tube_volume' ]
df_test_merged[ 'bend_num_X_volume' ] = df_test_merged[ 'num_bends' ].replace( 0, 1 ) * df_test_merged[ 'tube_volume' ]

df_train_merged[ 'bend_num_X_length' ] = df_train_merged[ 'num_bends' ].replace( 0, 1 ) * df_train_merged[ 'length' ]
df_test_merged[ 'bend_num_X_length' ] = df_test_merged[ 'num_bends' ].replace( 0, 1 ) * df_test_merged[ 'length' ]

df_train_merged[ 'bend_num_X_diameter' ] = df_train_merged[ 'num_bends' ].replace( 0, 1 ) * df_train_merged[ 'diameter' ]
df_test_merged[ 'bend_num_X_diameter' ] = df_test_merged[ 'num_bends' ].replace( 0, 1 ) * df_test_merged[ 'diameter' ]

df_train_merged[ 'bend_num_X_wall' ] = df_train_merged[ 'num_bends' ].replace( 0, 1 ) * df_train_merged[ 'wall' ]
df_test_merged[ 'bend_num_X_wall' ] = df_test_merged[ 'num_bends' ].replace( 0, 1 ) * df_test_merged[ 'wall' ]

# df_train_merged[ 'bend_num_X_quantity' ] = df_train_merged[ 'num_bends' ].replace( 0, 1 ) * df_train_merged[ 'quantity' ]
# df_test_merged[ 'bend_num_X_quantity' ] = df_test_merged[ 'num_bends' ].replace( 0, 1 ) * df_test_merged[ 'quantity' ]

# df_train_merged[ 'bend_num_by_radius_x_volume' ] = df_train_merged[ 'bend_num_by_radius' ].replace( 0, 1 ) * df_train_merged[ 'tube_volume' ]
# df_test_merged[ 'bend_num_by_radius_x_volume' ] = df_test_merged[ 'bend_num_by_radius' ].replace( 0, 1 ) * df_test_merged[ 'tube_volume' ]

In [94]:
# # quantity 관계고려하여 feature value 조정
# for i in range( 1, 9 ) :
#     df_train_merged[ 'quantity_' + str( i ) ] = df_train_merged[ 'quantity_' + str( i ) ] * df_train_merged[ 'quantity' ]
#     df_test_merged[ 'quantity_' + str( i ) ] = df_test_merged[ 'quantity_' + str( i ) ] * df_test_merged[ 'quantity' ]
    
# list_to_multiply_with_quantity = [
#     'weight_sum',

#     'uniqueness_count',
#     'orientation_count',

#     'adaptor_sum',
#     'boss_sum',
#     'elbow_sum',
#     'float_sum',
#     'hfl_sum',
#     'nut_sum',
#     'other_sum',
#     'sleeve_sum',
#     'straight_sum',
#     'tee_sum',
#     'threaded_sum',

#     'adaptor_weight_sum',
#     'boss_weight_sum',
#     'elbow_weight_sum',
#     'float_weight_sum',
#     'hfl_weight_sum',
#     'nut_weight_sum',
#     'other_weight_sum',
#     'sleeve_weight_sum',
#     'straight_weight_sum',
#     'tee_weight_sum',
#     'threaded_weight_sum',

#     'comp_type_count',
#     'comp_total_count',

#     'tube_volume',

#     'spec_type_count',

#     'req_comp_spec_type_count',
#     'req_comp_spec_total_count',
# ]

# for t in list_to_multiply_with_quantity :
#     df_train_merged[ t ] = df_train_merged[ t ] * df_train_merged[ 'quantity' ]
#     df_test_merged[ t ] = df_test_merged[ t ] * df_test_merged[ 'quantity' ]

In [95]:
list_to_log = [ 'tube_volume', 'bend_num_X_volume', 'bend_num_X_length', 'bend_num_X_diameter', 'bend_num_X_wall',
#                 'bend_num_X_quantity',    # 좋지않음
#                 'bend_num_by_radius_x_volume',    # 좋지않음
#                 'supplier_total_sales_result', 'supplier_annual_sales_result',    # 좋지않음
#                 'material_total_sales_result', 'material_annual_sales_result',    # 좋지않음
              ]

# log함수 적용 테스트
# X[ 'tube_volume' ] = np.log10( X[ 'tube_volume' ] )
# df_test_merged[ 'tube_volume' ] = np.log10( df_test_merged[ 'tube_volume' ] )

# df_train_merged[ 'bend_num_by_radius_x_volume' ] = df_train_merged[ 'bend_num_by_radius' ].replace( 0, 1 ) * df_train_merged[ 'tube_volume' ]
# df_test_merged[ 'bend_num_by_radius_x_volume' ] = df_test_merged[ 'bend_num_by_radius' ].replace( 0, 1 ) * df_test_merged[ 'tube_volume' ]

for feat in list_to_log :
    df_train_merged[ feat ].replace( to_replace = 0, value = 1, inplace = True )
    df_test_merged[ feat ].replace( to_replace = 0, value = 1, inplace = True )
    
    df_train_merged[ feat ] = np.log10( df_train_merged[ feat ] )
    df_test_merged[ feat ] = np.log10( df_test_merged[ feat ] )

In [96]:
if bracket != 'All' :
    df_train_merged.drop( 'bracket_pricing', axis = 1, inplace = False )
    df_test_merged.drop( 'bracket_pricing', axis = 1, inplace = False )

##4.11 Prediction Modeling

In [97]:
X = df_train_merged.drop( 'cost', axis = 1, inplace = False )    # X를 확보
y = np.log1p( df_train_merged[ 'cost' ] )   # y를 확보

del( df_train )
del( df_train_merged )

In [98]:
cv_cnt = 10    # cv : cross validation 횟수
n_jobs_cnt = 6

In [99]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn import grid_search

In [100]:
if mode == True :
    model_list = [
#                    RandomForestRegressor(),    # 제출용
                   GradientBoostingRegressor( n_estimators = 2500,
                                              max_depth = 7,
                                              warm_start = True,
                                              subsample = 0.8
                                            ),
                 ]
else :
    model_list = [
#                    RandomForestRegressor(),    # 테스트용
                   GradientBoostingRegressor(),    # 테스트용
                 ]    

In [101]:
%%time
import time
for model in model_list :
    model_name = str( model ).split( '(' )[0]    # 파일생성용
    
    if model_name.startswith( 'GradientBoostingRegressor' ) :
        params = {
    #                'n_estimators' : (1500,2000,2500,3000),
    #                'max_depth' : (5,7,9),
#                    'subsample' : (1.0,0.9,0.8,0.7)
                 }
    elif model_name.startswith( 'RandomForestRegressor' ) :
        params = {
                 }        

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params,
                                   n_jobs = n_jobs_cnt,
                                   cv = cv_cnt,
                                 )
    gs.fit( X, y )
    
    print( '=========================================' )
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )
    
    df_feature_importance = pd.DataFrame( X.columns.values, columns = [ 'features' ] )
    df_feature_importance[ 'importance' ] = gs.best_estimator_.feature_importances_ * 100
    print( df_feature_importance.sort( 'importance', ascending = False  ) )    
    
    
    y_pred = gs.best_estimator_.predict( df_test_merged )    # prediction 수행
    df_result[ 'cost' ] = np.expm1( y_pred )
    
    now = time.strftime( '%Y%m%d%H%M%S' )    # 현재시각을 확보
    file_timestamp = now[2:4] + now[4:6] + now[6:8] + now[8:10] + now[10:12] + now[12:14]
    accuracy = '{0:.1f}%'.format( gs.best_score_ * 100 )    # latitude에 대한 예측률 저장 : 파일명 활용용도임
    
    if bracket == 'Yes' :
        df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                          '.' + accuracy + '.result().bracket.Y.csv', sep = ',', index = False )
    elif bracket == 'No' :
        df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                          '.' + accuracy + '.result().bracket.N.csv', sep = ',', index = False )
    elif bracket == 'All' :
        df_result.to_csv( path_or_buf = './result.to.submit/' + file_timestamp + '.' + model_name +
                          '.' + accuracy + '.result().bracket.All.csv', sep = ',', index = False )                
    
    
    list_features = df_test_merged.columns.values.tolist()    
    
    with open( 'performance.condition.history.txt', 'a' ) as history_file :    # 모델의 이력을 logging
        history_file.write( '======================================================================\n' )
        history_file.write( '0. Description :\n' )
        if mode == True :
            history_file.write( '\tmode : Production mode\n' )
        else :
            history_file.write( '\tmode : Test mode\n' )
        history_file.write( '1. File name :\n\t' + file_timestamp + '.' + model_name + '.' + accuracy + '\n' )
        history_file.write( '2. Model information :\n\t' + str( gs.best_estimator_ ) + '\n' )
        history_file.write( '3. Applied features :\n' )
        for feature in list_features :
            history_file.write( '\t' + feature + '\n' )
        history_file.write( '4. Removed features :\n' )
        for feature in list_for_remove :
            history_file.write( '\t' + feature + '\n' )
        history_file.write( '5. Feature importance :\n\t' )        
        history_file.write( df_feature_importance.sort( 'importance', ascending = False  ).to_string() )
        history_file.write( '\n' )
        history_file.close()    

model :  GradientBoostingRegressor
best_score :  0.775010664134
best model :  GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=7, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=2500,
             random_state=None, subsample=0.8, verbose=0, warm_start=True)
                      features  importance
70                       spec2   11.586090
189            bend_num_X_wall    9.642076
186          bend_num_X_volume    9.041565
187          bend_num_X_length    8.459423
8                         date    7.881868
7                          day    6.692056
29              component_id_3    6.432663
68                 tube_volume    3.675587
12                      length    2.815673
9                  material_id    2.376031
25              component_id_1    2.158150
183               sales_result    2.121031
41                  weight_s

In [102]:
df_result.head( 3 )

,id,cost
40,41,7.996248
41,42,48.591876
54,55,7.892302


In [103]:
df_result_desc = df_result.describe()    # 결과의 overview 확인
df_result_desc.drop( 'id', axis = 1, inplace = True )
df_result_desc.describe()

,cost
count,8.000000
mean,616.002910
std,1383.602416
min,0.495316
25%,6.959094
50%,17.925937
75%,249.967230
max,3956.000000


In [104]:
del( X )
del( y )
del( df_test )
del( df_test_merged )
del( df_components )
del( df_result_desc )
del( df_result )